# Worksheet 08

Name:  Thian Amarasekera

UID: U91933393 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.098804315808113, 5.678021418602914, 4.827619427234815, 4.425005844243177, 5.240624422606968, 3.428903947779625, 5.0131778002417136, 5.649031876614431, 4.468864331256134, 5.969781907994889]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[8.209501038981761, 9.548671561802763, 8.072924929179573, 7.247392859464998, 7.68558899188849, 8.855066825500836, 8.74466808172594, 7.649015991705814, 8.754631518862457, 9.12357729605894]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.969781907994889, 4.468864331256134, 9.12357729605894, 5.649031876614431, 5.0131778002417136, 8.754631518862457, 3.428903947779625, 5.240624422606968, 7.649015991705814, 8.74466808172594]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.


We know the mean, variance, and the probability of each component. For c1, the mean is 5, the variance is 1, and the probability is 0.5. For c2, the mean is 8, the variance is 1, and the probability is 0.5.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.969781907994889, 4.468864331256134, 5.649031876614431, 5.0131778002417136, 3.428903947779625, 5.240624422606968]
[9.12357729605894, 8.754631518862457, 7.649015991705814, 8.74466808172594]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 4.961730714415626,  mean_2 = 8.567973222088288
var_1 = 0.6935756279516027,  var_2 = 0.30481016786550424


The actual value for mean is 5 and 8, and the estimated value is 4.9 and 8.1. They are close to the true values. The actual value for variance is 1 and 1, and the estimated value is 0.51 and 0.58, which is not really close. The actual value for P(C_j) is 0.5 and 0.5, and the estimated value is 0.7 and 0.3, which is not as close.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  5.969781907994889
probability of observing that point if it came from cluster 0 =  0.2000382116280434
probability of observing that point if it came from cluster 1 =  2.1847387819683278e-16
point =  4.468864331256134
probability of observing that point if it came from cluster 0 =  0.44685034192065115
probability of observing that point if it came from cluster 1 =  7.009171723460944e-40
point =  9.12357729605894
probability of observing that point if it came from cluster 0 =  8.730511613103304e-09
probability of observing that point if it came from cluster 1 =  0.24854060803958478
point =  5.649031876614431
probability of observing that point if it came from cluster 0 =  0.3520303407334718
probability of observing that point if it came from cluster 1 =  1.5975223725750586e-20
point =  5.0131778002417136
probability of observing that point if it came from cluster 0 =  0.5736162702182198
probability of observing that point if it came from cluster 1 =  3.825775642969272e-30
point 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6032988611214593,  P(C_2) = 0.3967011388785408
mean_1 = 4.976425001257847,  mean_2 = 8.575614860563995
var_1 = 0.7290553653040548,  var_2 = 0.3002641172083457


The mean_j is 4.4 and 8.1, which is close to the original value. The variance_j is 0.51 and 0.58, which is not close to the original value. The P(C_j) is 0.7 and 0.3, which is not close to the original value.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_c0_x_new = [] # P(C_0 | X_i)
prob_c1_x_new = [] # P(C_1 | X_i)

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x_new.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x_new.append( pdf_i[1] * prob_c[1] / prob_x )

print("Old probabilities(c0): ", prob_c0_x)
print("New probabilities(c0): ", prob_c0_x_new)
print("Old probabilities(c1): ", prob_c1_x)
print("New probabilities(c1): ", prob_c1_x_new)

point =  5.969781907994889
probability of observing that point if it came from cluster 0 =  0.2162821420108036
probability of observing that point if it came from cluster 1 =  5.871941336744321e-17
point =  4.468864331256134
probability of observing that point if it came from cluster 0 =  0.42944018197368145
probability of observing that point if it came from cluster 1 =  3.1843196197645003e-41
point =  9.12357729605894
probability of observing that point if it came from cluster 0 =  5.149240671163645e-08
probability of observing that point if it came from cluster 1 =  0.2513172390882416
point =  5.649031876614431
probability of observing that point if it came from cluster 0 =  0.3575416209307948
probability of observing that point if it came from cluster 1 =  3.124900867002022e-21
point =  5.0131778002417136
probability of observing that point if it came from cluster 0 =  0.5465094981122259
probability of observing that point if it came from cluster 1 =  3.6072715992567588e-31
point =

There is not much of a difference between the updated P(C_j | X_i) and the original.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
assignment = []

for i in range(len(data)):
    if prob_c0_x_new[i] > prob_c1_x_new[i]:
        assignment.append(0)
    else:
        assignment.append(1)

print("Data Point       | Cluster Assignment")
for data_point, cluster in zip(data, assignment):
    print(f"{data_point} | {cluster}")

Data Point       | Cluster Assignment
5.969781907994889 | 0
4.468864331256134 | 0
9.12357729605894 | 1
5.649031876614431 | 0
5.0131778002417136 | 0
8.754631518862457 | 1
3.428903947779625 | 0
5.240624422606968 | 0
7.649015991705814 | 1
8.74466808172594 | 1


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      no       |
| A  C |      yes      |
| A  D |      yes      |
| A  E |      yes      |
| B  C |      yes      |
| B  D |      yes      |
| B  E |      yes      |
| C  D |      yes      |
| C  E |      yes      |
| D  E |       no      |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

 (N C 2) = (N!)/2!(N-2)! 

 (N C 2) = (N(N - 1))/ 2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

Disagreements = 3(3-1)/2 = 3

Disagreements = 2(2-1)/2 = 2

Disagreements = 4(4-1)/2 = 6

Total Disagreements = 3 + 1 + 6 = 10

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

Agreements = (3 x (3-1))/2 = 3

Agreements = (2 x (2-1))/2 = 1

Agreements = (4 x (4-1))/2 = 6

So, the total number of agreements is 36 - 10 = 26

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

1. extract the data points in each distinct cluster
2. compute the number of disagreements in each cluster by comparing all pairs of points in that cluster and checking whether they agree or disagree
3. compute the total number of disagreements